# Price experiment A/B test

`price_experiment.csv` dataset contains transaction recordsreceived during testing different product price configurations.

- `user_conversion_rate`
- `ARPO` - average revenue per paid order (price)
- `AOPU` - average number of paid orders per user
- `ret1to2` - ratio of users who retuned to service and bought their 2nd order after buying the 1st
- `ARPU_visitor` - average revenue per user (visitor), without excluding `order_costs`
- `APPU_visitor` - average profit per user (visitor), excluding `order_costs`


The Question: Which price configuration `Default` or `New` would you recommend to use in future? Why?

In [1]:
import pandas as pd

df = pd.read_csv("price_experiment.csv", index_col=0, parse_dates=["order_date"])
df.head(10)

,order_date,customer_id,is_paid_order,order_total_usd,order_costs_usd,split_variant
order_id,,,,,,
0,2019-11-20 14:31:25,2688275,0,0.0,0.0,New
1,2019-11-20 14:32:38,2688282,0,0.0,0.0,New
2,2019-11-20 14:32:55,2688283,0,0.0,0.0,Default
3,2019-11-20 14:36:12,2688298,0,0.0,0.0,Default
4,2019-11-20 14:37:49,2688306,0,0.0,0.0,Default
5,2019-11-20 14:40:20,2688313,0,0.0,0.0,Default
6,2019-11-20 14:41:26,2688315,0,0.0,0.0,Default
7,2019-11-20 14:41:30,2688316,0,0.0,0.0,New
8,2019-11-20 14:55:00,2688345,0,0.0,0.0,New


In [0]:
# YOUR CODE GOES HERE

from scipy.stats import ttest_ind

grouped = df.groupby("customer_id")

df_usrs = pd.DataFrame()
df_usrs["OPU"] = grouped.is_paid_order.sum()
df_usrs["RPO"] = grouped.order_total_usd.mean()
df_usrs["RPU"] = grouped.order_total_usd.sum()
df_usrs["PPU"] = grouped.apply(lambda x: x.order_total_usd.sum() - x.order_costs_usd.sum())
df_usrs["split_variant"] = grouped.apply(lambda x: x.split_variant.iloc[0])

In [10]:
df_usrs

,OPU,RPO,RPU,PPU,split_variant
customer_id,,,,,
193785,1,226.696,226.696,157.896,Default
244553,0,0.000,0.000,0.000,New
446050,0,0.000,0.000,0.000,Default
494511,0,0.000,0.000,0.000,Default
512605,0,0.000,0.000,0.000,New
...,...,...,...,...,...
2759315,0,0.000,0.000,0.000,Default
2759317,0,0.000,0.000,0.000,New
2759331,1,61.576,61.576,28.896,Default


In [4]:
# user_conversion_rate

data1 = df_usrs[df_usrs.split_variant == "Default"].OPU.astype(bool).astype(int)
data2 = df_usrs[df_usrs.split_variant == "New"].OPU.astype(bool).astype(int)

_, pvalue = ttest_ind(data1, data2, nan_policy="omit")
pvalue /= 2  # two-tailed ===>> one-tailed test

print("user_conversion_rate")
print("---------------------")
print(f"Default: {data1.mean(): .3f},  New: {data2.mean(): .3f}")
print(f"Relative uplift: {(data2.mean() - data1.mean())/data1.mean()*100: .1f}%")
print(f"p-value: {pvalue: .3f}")

user_conversion_rate
---------------------
Default:  0.080,  New:  0.069
Relative uplift: -13.7%
p-value:  0.020


In [5]:
(data1 > 1).sum()

0

In [6]:
# ARPO

mask_paid = (df_usrs.OPU >= 1)
mask_split = (df_usrs.split_variant == "Default")

data1 = df_usrs[mask_paid & mask_split].RPO
data2 = df_usrs[mask_paid & ~mask_split].RPO

_, pvalue = ttest_ind(data1, data2, nan_policy="omit")
pvalue /= 2  # two-tailed ===>> one-tailed test

print("ARPO")
print("---------------------")
print(f"Default: {data1.mean(): .3f},  New: {data2.mean(): .3f}")
print(f"Relative uplift: {(data2.mean() - data1.mean())/data1.mean()*100: .1f}%")
print(f"p-value: {pvalue: .3f}")

ARPO
---------------------
Default:  220.073,  New:  247.558
Relative uplift:  12.5%
p-value:  0.020


In [7]:
# AOPU

mask_paid = (df_usrs.OPU >= 1)
mask_split = (df_usrs.split_variant == "Default")

data1 = df_usrs[mask_paid & mask_split].OPU
data2 = df_usrs[mask_paid & ~mask_split].OPU

_, pvalue = ttest_ind(data1, data2, nan_policy="omit")
pvalue /= 2  # two-tailed ===>> one-tailed test

print("AOPU")
print("---------------------")
print(f"Default: {data1.mean(): .3f},  New: {data2.mean(): .3f}")
print(f"Relative uplift: {(data2.mean() - data1.mean())/data1.mean()*100: .1f}%")
print(f"p-value: {pvalue: .3f}")

AOPU
---------------------
Default:  1.204,  New:  1.299
Relative uplift:  7.9%
p-value:  0.071


In [9]:
data1

customer_id
193785     0
2260394    0
2268971    0
2369820    0
2393876    0
          ..
2758410    0
2758760    0
2758995    0
2759101    0
2759331    0
Name: OPU, Length: 387, dtype: int64

In [8]:
# ret1to2

mask_paid = (df_usrs.OPU >= 1)
mask_split = (df_usrs.split_variant == "Default")

data1 = df_usrs[mask_paid & mask_split].OPU.apply(lambda x: (x >= 2)).astype(int)
data2 = df_usrs[mask_paid & ~mask_split].OPU.apply(lambda x: (x >= 2)).astype(int)

_, pvalue = ttest_ind(data1, data2, nan_policy="omit")
pvalue /= 2  # two-tailed ===>> one-tailed test

print("AOPU")
print("---------------------")
print(f"Default: {data1.mean(): .3f},  New: {data2.mean(): .3f}")
print(f"Relative uplift: {(data2.mean() - data1.mean())/data1.mean()*100: .1f}%")
print(f"p-value: {pvalue: .3f}")

AOPU
---------------------
Default:  0.119,  New:  0.186
Relative uplift:  56.2%
p-value:  0.006


In [0]:
# ARPU_visitor

mask_split = (df_usrs.split_variant == "Default")

data1 = df_usrs[mask_split].RPU
data2 = df_usrs[~mask_split].RPU

_, pvalue = ttest_ind(data1, data2, nan_policy="omit")
pvalue /= 2  # two-tailed ===>> one-tailed test

print("ARPU_visitor")
print("---------------------")
print(f"Default: {data1.mean(): .3f},  New: {data2.mean(): .3f}")
print(f"Relative uplift: {(data2.mean() - data1.mean())/data1.mean()*100: .1f}%")
print(f"p-value: {pvalue: .3f}")

ARPU_visitor
---------------------
Default:  26.805,  New:  27.333
Relative uplift:  2.0%
p-value:  0.425


In [0]:
# APPU_visitor

mask_split = (df_usrs.split_variant == "Default")

data1 = df_usrs[mask_split].PPU
data2 = df_usrs[~mask_split].PPU

_, pvalue = ttest_ind(data1, data2, nan_policy="omit")
pvalue /= 2  # two-tailed ===>> one-tailed test

print("APPU_visitor")
print("---------------------")
print(f"Default: {data1.mean(): .3f},  New: {data2.mean(): .3f}")
print(f"Relative uplift: {(data2.mean() - data1.mean())/data1.mean()*100: .1f}%")
print(f"p-value: {pvalue: .3f}")

APPU_visitor
---------------------
Default:  17.461,  New:  19.098
Relative uplift:  9.4%
p-value:  0.211


Total profit per visitor increased despite the conversion drop. However results are not significat yet, so i would continue the experiment.